In [1]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings2 # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy


# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
     def __init__(self, csocket):
        self.client_socket = csocket
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings2.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False


In [2]:
print(settings2.TABLE_NAME)

maythe4th


In [3]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [4]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="iembd2019",
    database="TwitterDB",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings2.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings2.TABLE_NAME, settings2.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()
    



In [5]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOEKN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [7]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings2.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

RT @iingwen: #MayThe4th be with all of those fighting to keep us safe from #COVID19. No matter what country or galaxy you’re from, we’re al…
Long: None, Lati: None
RT @OnePerfectShot: THE MANDALORIAN (2019)

Cinematography by Greig Fraser
Directed by Dave Filoni
From 'Chapter 1: The Mandalorian'
No. 7…
Long: None, Lati: None
RT @PFWRIGHT: Happy #MayThe4th be with you! @PFWRIGHT
Long: None, Lati: None
RT @nilosophy: May the 4th be with you.

@BTS_twt @bts_bighit 
#민윤기 #BTS #MayThe4th https://t.co/gu8OJ1uoms
Long: None, Lati: None
#MayThe4th I don't care for starwars, I just fucking like the day
Long: None, Lati: None
RT @uzuriartonline: Happy May the Fourth!
@starwars 
#art #artist #StarWarsDay #StarWars #StarWarsfanart #MayThe4th #fanart https://t.co/xX…
Long: None, Lati: None
Aaah - there it is! Call off the attack!! 
#ReturnoftheJedi #MayThe4th #ECCC https://t.co/JQJS1bW0jL
Long: None, Lati: None
RT @Calvinn_Hobbes: In honour of #MayThe4thBeWithYou #MayThe4th here are some amazing Ca

RT @ADAMDRIVERE: in honour of #MayThe4th i present— qui gon and obi wan fighting darth maul but to gimmie gimmie gimmie by abba https://t.c…
Long: None, Lati: None
RT @JenniferLanda: Happy #MayThe4th! This is what Star Wars Day means to me. https://t.co/i9mLnsEAVc
Long: None, Lati: None
RT @TheRedwood2: #MayThe4th cumming at y’all! All day! Collaborating with some more of my sexy friends! Beautiful! Creative! #StarWars fans…
Long: None, Lati: None
RT @OnePerfectShot: STAR WARS: A NEW HOPE (1977)

Cinematography by Gilbert Taylor
Directed by George Lucas
No. 1 on our #StarWars100 for #…
Long: None, Lati: None
RT @OJessicaNigri: 🖤MAY THE 4th BE WITH YOU!🖤

#MayThe4thBeWithYou #MayThe4th https://t.co/LI8zOVV5NK
Long: None, Lati: None
RT @StarWarsNewsNet: Are you happy with the news that Taika Waititi is directing an upcoming #StarWars feature film? #MayThe4th
Long: None, Lati: None
RT @LAFC: This Is The Way. 

#MayThe4th | @LAFC3252 https://t.co/S8Oj1OuC79
Long: None, Lati: None
RT @OnePe

RT @ZugMichael: #MayThe4thBeWithYou  ... meets #MerMay in this little piece that nobody asked for and even fewer want to see.  #StarWars  #…
Long: None, Lati: None
RT @Covers: C-3PO is gambling Twitter. 

Han Solo is all of us rolling the dice on a 6-team parlay. Again.

Happy #MayThe4th folks. https:/…
Long: None, Lati: None
RT @OnePerfectShot: STAR WARS: THE LAST JEDI (2017)

Cinematography by Steve Yedlin
Directed by Rian Johnson
No. 2 on our #StarWars100 for…
Long: None, Lati: None
RT @Kre8tur: Passengers in Lagos should write down the plate number of every bus they enter today,

So as NCDC announce, Danfo ZPXXU carrie…
Long: None, Lati: None
But then another Ewok adventure set things right! 

Happy #MayThe4th everyone! https://t.co/itSFaQA5mR
Long: None, Lati: None
RT @SmithsonianMag: So many things come in threes, even Star Wars trilogies. #MayThe4th @airandspace https://t.co/kIBZnPRMGI
Long: None, Lati: None
RT @LSWGame: I + II + III + IV + V + VI + VII + VIII + IX = LEGO Star W

RT @Rodriguez: I am truly humbled to say I have now had the very rare privilege of directing the biggest star in the universe. @StarWars #T…
Long: None, Lati: None
RT @LSWGame: I + II + III + IV + V + VI + VII + VIII + IX = LEGO Star Wars: The Skywalker Saga. All 9 films in one massive game! #Maythe4th…
Long: None, Lati: None
RT @fayren: It's #MayThe4th! Din dressing up for the occasion. https://t.co/DzNiC5czta
Long: None, Lati: None
RT @OnePerfectShot: STAR WARS: A NEW HOPE (1977)

Cinematography by Gilbert Taylor
Directed by George Lucas
No. 1 on our #StarWars100 for #…
Long: None, Lati: None
RT @fayren: It's #MayThe4th! Din dressing up for the occasion. https://t.co/DzNiC5czta
Long: None, Lati: None
RT @Bulbagarden: Happy #MayThe4th! Would you watch a Star Wars crossover with Pokémon? https://t.co/SPOHRmvYOi
Long: None, Lati: None
RT @OnePerfectShot: STAR WARS: THE EMPIRE STRIKES BACK (1980)

Cinematography by Peter Suschitzky
Directed by Irvin Kershner
No. 2 on our #…
Long: None, L

RT @OnePerfectShot: STAR WARS: THE EMPIRE STRIKES BACK (1980)

Cinematography by Peter Suschitzky
Directed by Irvin Kershner
No. 33 on our…
Long: None, Lati: None
RT @OnePerfectShot: STAR WARS: THE LAST JEDI (2017)

Cinematography by Steve Yedlin
Directed by Rian Johnson
No. 2 on our #StarWars100 for…
Long: None, Lati: None
RT @daztibbles: Concept and Final (cropped) of my #MayThe4th artwork. @starwars 
#StarWarsFanArt #MayThe4thBeWithYou https://t.co/lt31bdHGFj
Long: None, Lati: None
RT @OnePerfectShot: STAR WARS: A NEW HOPE (1977)

Cinematography by Gilbert Taylor
Directed by George Lucas
No. 5 on our #StarWars100 for #…
Long: None, Lati: None
RT @Rodriguez: I am truly humbled to say I have now had the very rare privilege of directing the biggest star in the universe. @StarWars #T…
Long: None, Lati: None
RT @theshrillest: how on earth is this a "perfect shot" https://t.co/K6vmrze5jX
Long: None, Lati: None
RT @cracksh0t: Just under the wire for #Maythe4th, here's a cover of @pswapninj

...

RT @TSRachaelBelle: Can I keep you pure, or can I tempt you to the dark side ;) Happy May the Fourth. #MayThe4th https://t.co/7dpEuLYjNl
Long: None, Lati: None
RT @OnePerfectShot: STAR WARS: THE EMPIRE STRIKES BACK (1980)

Cinematography by Peter Suschitzky
Directed by Irvin Kershner
Production Des…
Long: None, Lati: None
RT @LStewart_books: Nice to see ladybirds out and about #ladybug #ladybird #insect #NaturePhotography #Spring #Earth #MayThe4th #mondaythou…
Long: None, Lati: None
RT @holidaytimeuk: Don't mind spending my quarantine here!

@holidaytimeuk
@TravelCuddly
@Stop_Trump20
@vucicaaaa
@DaiIyLuxury
@travelust_b…
Long: None, Lati: None
To my surprise, they were adequately entertained by Episode I. Didn’t give a crap about Jar Jar but liked the actio… https://t.co/yRxOf0XLAd
Long: None, Lati: None
RT @KaijuNewsOutlet: Once again, happy #MayThe4th to everyone. https://t.co/7MnE2ttrFj
Long: None, Lati: None
RT @OnePerfectShot: STAR WARS: THE EMPIRE STRIKES BACK (1980)

Cinematogra

RT @HeavyMetalInk: May the 4th? Happy Star Wars Day... but this ain't Star Wars. Not exactly. Who remembers @RodKierkegaard "Rock Opera?" L…
Long: None, Lati: None
RT @EricCanete: Sometimes...Tommy makes it hard to concentrate.
Long: None, Lati: None
RT @rianjohnson: ahhhh FEEL THE LOVE #MayThe4th https://t.co/W2MSiZlv0r
Long: None, Lati: None
elon musk really is the king of the nerds  having his kid on #MayThe4th
Long: None, Lati: None
RT @OnePerfectShot: ROGUE ONE: A STAR WARS STORY (2016)

Cinematography by Greig Fraser
Directed by Gareth Edwards
No. 6 on our #StarWars10…
Long: None, Lati: None
RT @OJessicaNigri: 🖤MAY THE 4th BE WITH YOU!🖤

#MayThe4thBeWithYou #MayThe4th https://t.co/LI8zOVV5NK
Long: None, Lati: None
this is it the good stuff
Long: None, Lati: None
RT @OnePerfectShot: STAR WARS: THE EMPIRE STRIKES BACK (1980)

Cinematography by Peter Suschitzky
Directed by Irvin Kershner
Production Des…
Long: None, Lati: None
RT @HGilroy67: #CloneWars fun fact! How'd Hondo Ohnaka ge

RT @SuperMoviePod: Happy #MayThe4th. We have a brand new #MovieCocktail to celebrate #StarWars. It’s the 15th Anniversary of #RevengeOfTheS…
Long: None, Lati: None
RT @OnePerfectShot: STAR WARS: THE FORCE AWAKENS (2015)

Cinematography by Dan Mindel
Directed by J.J. Abrams
No. 31 on our #StarWars100 fo…
Long: None, Lati: None
@elonmusk and @Grimezsz just became parents of a baby Jedi.

I’m thrilled!!! It’s a super space family!. 

Love and… https://t.co/Rbit0FFnxY
Long: None, Lati: None
RT @OnePerfectShot: STAR WARS: RETURN OF THE JEDI (1983)

Cinematography by Alan Hume and Alec Mills
Directed by Richard Marquand
Productio…
Long: None, Lati: None
I picked a side #MayThe4th https://t.co/ocu5mn1zuA
Long: None, Lati: None
RT @hitRECordJoe: The @starwars behind the scenes photo we all need right now.. 💙 #MayThe4th https://t.co/t2jnnZhblh
Long: None, Lati: None
RT @Giancarlo_Volpe: In 2005 Dave Filoni &amp; I cosplayed as Kit Fisto &amp; Plo Koon, two random Jedi we thought were cool &amp;

RT @Newegg: To celebrate #Maythe4th, we're giving away this interstellar custom PC built by @robeytech &amp; @halogenica ft. a handcrafted + 3D…
Long: None, Lati: None
RT @Rodriguez: I am truly humbled to say I have now had the very rare privilege of directing the biggest star in the universe. @StarWars #T…
Long: None, Lati: None
Can’t get enough of the Star Wars Day? Keep the celebrations going with the Best of Star Wars playlist - all your f… https://t.co/zNJQ4QnGNy
Long: None, Lati: None
RT @OnePerfectShot: STAR WARS: REVENGE OF THE SITH (2005)

Cinematography by David Tattersall
Directed by George Lucas
No. 67 on our #StarW…
Long: None, Lati: None
RT @LinusTech: I showed u my lightsaber pls respond #MayThe4th https://t.co/SHju7fwRYi
Long: None, Lati: None
RT @CommodoreBlog: It's time..
#MayThe4th #StarWarsDay https://t.co/4mgvMZVMx1
Long: None, Lati: None
RT @quiethandfilms: Look what I just received in the mail!!!! I think that’s me?!!!! And I keep hearing something is special abo

KeyboardInterrupt: 